In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_plot
import gta_prices
import gta_signals
import gta_statistics
import gta_trades
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
look_back = (p1, p2, p3, p4)
strategy = f'Donchian'
position_size = 7500  # Position size in major currency unit.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
# Import weekly closing prices.
df = gta_prices.weekly(exchange, tidm, history_start)

In [4]:
# Calculate Donchian channel for each System.
for period in look_back:
    sys = look_back.index(period) + 1
    dc = gta_indicator.donchian(df, sys, period)
    df = df.join(dc)

In [5]:
# System 1 long trades.

# Check for close above upper Donchian channel.
df['bto1'] = np.where(df.close > df.upr1, 1, 0)

# Check for close below mid Donchian channel.
df['stc1'] = np.where(df.close < df.mid1, 1, 0)

# Trade state signal.
df['ss1'] = gta_signals.state_signal(df.bto1, df.stc1, p1)

# Trade entry & exit flags.
df['en1'] = np.where(np.logical_and(df.ss1 == 1, df.ss1.shift(periods=1) == 0), 1, 0)
df['ex1'] = np.where(np.logical_and(df.ss1 == 0, df.ss1.shift(periods=1) == 1), 1, 0)

In [6]:
# System 2 long trades.

# Check for close below mid Donchian channel.
df['stc2'] = np.where(df.close < df.mid2, 1, 0)

# Trade state signal.
df['ss2'] = gta_signals.state_signal(df.bto1, df.stc2, p2)

# Trade exit flags.
df['ex2'] = np.where(np.logical_and(df.ss2 == 0, df.ss2.shift(periods=1) == 1), 1, 0)

In [7]:
# System 3 long trades.

# Check for close below mid Donchian channel.
df['stc3'] = np.where(df.close < df.mid3, 1, 0)

# Trade state signal.
df['ss3'] = gta_signals.state_signal(df.bto1, df.stc3, p2)

# Trade exit flags.
df['ex3'] = np.where(np.logical_and(df.ss3 == 0, df.ss3.shift(periods=1) == 1), 1, 0)

In [8]:
# Trade list.
td1 = gta_trades.trade_list(df.close, df.en1,  df.ex1)
for i in range(len(look_back) - 1):
    i += 1
    frame = gta_trades.trade_list(df.close, df.en1,  df[f'ex{i}'])
    td = pd.concat([td1, frame], axis=0)
td = td.sort_index()
td

,entry_price,exit_date,exit_price
date,,,
2011-08-26,2.260,2013-08-02,3.51
2011-08-26,2.260,2012-06-01,3.45
2016-05-13,0.420,2017-07-21,0.66
2016-05-13,0.420,2017-02-17,0.82
2018-09-07,0.825,2020-02-28,2.17
2018-09-07,0.825,2019-10-11,2.75
2020-11-20,2.880,2021-05-07,2.97
2020-11-20,2.880,2021-01-29,3.15
2022-04-15,4.230,2022-04-22,4.28
